### Download the DEM automatically from USGS

https://apps.nationalmap.gov/downloader/

An arc second is equivalent to 1/3600th of 1 degree or equal to 1/1,296,000 of a full circle.

- 1-arc-second (~30m)	`Elevation/1/TIFF/current/`
- 1/3-arc-second (~10m)	`Elevation/13/TIFF/current/`
- 1/9-arc-second (~3m)	`Elevation/19/TIFF/current/`

- 1m resolution DEM  `https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1m/TIFF/current/{tile_name}/USGS_1m_{tile_name}.tif`

`n40w076` → Represents the latitude (40°N) and longitude (76°W) of the bottom-left corner. `USGS_1_n40w076.tif` → USGS 1-arcsecond DEM for that region. To get a different tile, change `n40w076` in the URL to the coordinates of your area of interest.

In [6]:
import requests

tile_name = 'n38w105'
tile_name = 'n39w076'
tile_name = 'n40w076'

# Example download URL for a tile, here the 1 mean 1-arc second DEM, resolution around 30m
tile_url = "https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/current/%s/USGS_1_%s.tif"%(tile_name, tile_name)

# Download the tile
response = requests.get(tile_url)

with open("%s.tif"%(tile_name), "wb") as file:
    file.write(response.content)

print("DEM tile downloaded: %s.tif"%(tile_name))


DEM tile downloaded: n40w076.tif


Here we can create a function that would return the tile name for any specified coordinate

In [56]:
def get_tile_name(lat, lon):
    """
    Returns the USGS tile name formatted correctly (e.g., 'n39w076').
    """
    lat_tile = f"n{int(abs(lat)):02d}" if lat >= 0 else f"s{int(abs(lat)):02d}"
    lon_tile = f"w{int(abs(lon)):03d}" if lon < 0 else f"e{int(abs(lon)):03d}"
    return f"{lat_tile}{lon_tile}"

# Example: Get tile for Philadelphia (39.95°N, 75.16°W)
lat, lon = 39.95, -75.16
tile_name = get_tile_name(lat, lon)
print("DEM Tile Name:", tile_name)

DEM Tile Name: n39w075


In [53]:
tile_name = 'n40w076'

# Example download URL for a tile, here the 1 mean 1-arc second DEM, resolution around 30m
tile_url = "https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/current/%s/USGS_1_%s.tif"%(tile_name, tile_name)

# Download the tile
response = requests.get(tile_url)

with open("%s.tif"%(tile_name), "wb") as file:
    file.write(response.content)

print("DEM tile downloaded: %s.tif"%(tile_name))

DEM tile downloaded: n38w075.tif


In [54]:
tile_url

'https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/current/n38w075/USGS_1_n38w075.tif'

Usually, we want to get the all potential tiles in a certain area, here we create a function to return all the tile names in a region

In [47]:
def get_tiles_in_bbox(min_lat, max_lat, min_lon, max_lon):
    """
    Returns a list of tile names covering a bounding box.
    """
    tile_list = []
    for lat in range(int(min_lat), int(max_lat) + 1):
        for lon in range(int(min_lon), int(max_lon) + 1):
            tile_list.append(get_tile_name(lat, lon))
    return tile_list

# Example: Get tiles for an area (Philadelphia to NYC region)
tiles = get_tiles_in_bbox(38, 41, -76, -74)
print("Tiles to Download:", tiles)

Tiles to Download: ['n38w076', 'n38w075', 'n38w074', 'n39w076', 'n39w075', 'n39w074', 'n40w076', 'n40w075', 'n40w074', 'n41w076', 'n41w075', 'n41w074']


Now we can start to download 

In [ ]:
import requests
import os

def download_dem_1arc(tile_name, save_path="."):
    """
    Downloads a 1-arc-second (~30m) DEM tile from USGS TNM.
    
    Parameters:
        tile_name (str): Tile name (e.g., "n39w076").
        save_path (str): Directory to save the DEM file.
    """
    base_url = "https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/current"
    url = f"{base_url}/{tile_name}/USGS_1_{tile_name}.tif"
    
    output_file = os.path.join(save_path, f"{tile_name}.tif")
    
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(output_file, "wb") as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)
        print(f"✅ DEM Tile {tile_name} downloaded successfully!")
    else:
        print(f"❌ Error: Tile {tile_name} not found. Check availability.")
        print(url)
# Download multiple tiles
save_directory = "DEM_Tiles"
os.makedirs(save_directory, exist_ok=True)  # Create folder if it doesn't exist

for tile in tiles:
    download_dem_1arc(tile, save_directory)

✅ DEM Tile n38w076 downloaded successfully!
❌ Error: Tile n38w075 not found. Check availability.
https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/current/n38w075/USGS_1_n38w075.tif
❌ Error: Tile n38w074 not found. Check availability.
https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/current/n38w074/USGS_1_n38w074.tif
✅ DEM Tile n39w076 downloaded successfully!
✅ DEM Tile n39w075 downloaded successfully!
❌ Error: Tile n39w074 not found. Check availability.
https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/current/n39w074/USGS_1_n39w074.tif
✅ DEM Tile n40w076 downloaded successfully!
✅ DEM Tile n40w075 downloaded successfully!
❌ Error: Tile n40w074 not found. Check availability.
https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/current/n40w074/USGS_1_n40w074.tif
✅ DEM Tile n41w076 downloaded successfully!
✅ DEM Tile n41w075 downloaded successfully!
✅ DEM Tile n41w074 downloaded successfully!


You may find some tiles are missing, this is because those tiles cover the ocean, and are not available in the USGS database.